In [68]:
## Problem Set 1
## ABM
## October 5th, 2022

In [69]:
import pandas as pd

In [70]:
df = pd.read_stata("../data/PS1_Data.dta")

### Question 1
Report sample statistics for the full sample, the balanced sub-panel, and the exiters

In [71]:
# Full Sample
df[['Y', 'L', 'I', 'K', 'A']].describe()

,Y,L,I,K,A
count,39569.000000,39569.000000,39569.000000,39569.000000,39569.000000
mean,13.487732,5.000947,5.034981,8.986055,8.539210
std,1.726792,1.001510,0.999631,1.872697,3.213174
min,5.913900,0.624850,1.128800,2.087800,1.000000
25%,12.424000,4.328900,4.367700,7.993000,6.000000
50%,13.591000,5.007000,5.031000,9.291300,9.000000
75%,14.664000,5.678400,5.712500,10.289000,11.000000
max,19.164000,8.861500,9.340300,14.572000,17.000000


In [72]:
# Creating a balanced sub-panel 
df['firm'].value_counts() # Can already see that some firms do not show up every period
df_balanced = df.groupby('firm').filter(lambda x : len(x) == 10)
df_balanced[['Y', 'L', 'I', 'K', 'A']].describe()

,Y,L,I,K,A
count,21800.000000,21800.000000,21800.000000,21800.000000,21800.000000
mean,13.407925,4.993716,5.044564,9.157427,7.320183
std,1.722635,1.003712,1.003588,1.804151,3.232118
min,5.913900,1.102600,1.128800,2.237400,1.000000
25%,12.357000,4.321175,4.367300,8.259625,5.000000
50%,13.515000,5.002050,5.041900,9.431800,7.000000
75%,14.571250,5.670400,5.726675,10.390000,10.000000
max,18.872000,8.861500,9.340300,14.335000,16.000000


In [73]:
# Exiters panel 
df_exiters = df.groupby('firm').filter(lambda x : len(x) < 10)
df_exiters[['Y', 'L', 'I', 'K', 'A']].describe()


,Y,L,I,K,A
count,17769.000000,17769.000000,17769.000000,17769.000000,17769.000000
mean,13.585644,5.009822,5.023223,8.775805,10.034780
std,1.726896,0.998759,0.994656,1.932915,2.472889
min,6.706300,0.624850,1.342600,2.087800,1.000000
25%,12.510000,4.341000,4.368300,7.660300,8.000000
50%,13.686000,5.012700,5.015400,9.105100,10.000000
75%,14.772000,5.689400,5.699600,10.151000,12.000000
max,19.164000,8.596500,8.870100,14.572000,17.000000


I can see that mean output and labor are slightly higher for exiters. Age of the firm is significantly higher for exiters. Capital, on the other hand, is higher for survivers. 

### Question 2
Using only the balanced sub-panel, compute total, between and within and random effects estimators for the equation below. Perform a Hausman test of RE vs FE.  

$$y_{it} = \beta_0 + \beta_a a_{it} + \beta_k k_{it} + \beta_l l_{it} + \gamma_t + \omega_{it} + \eta_{it}$$

In [74]:
# Adjust dataset
import statsmodels.api as sm

year = pd.Categorical(df_balanced.year)
df_balanced = df_balanced.set_index(['firm', 'year'])
df_balanced["year"] = year

In [75]:
# Pooled OLS
from linearmodels import PooledOLS

exog_vars = ["A", "K", "L", "year"] 
exog = sm.add_constant(df_balanced[exog_vars]) # this adds a constant 

mod = PooledOLS(df_balanced.Y, exog) # this automatically excludes dummy year = 1
pooled_res = mod.fit()
print(pooled_res)

                          PooledOLS Estimation Summary                          
Dep. Variable:                      Y   R-squared:                        0.5270
Estimator:                  PooledOLS   R-squared (Between):              0.4762
No. Observations:               21800   R-squared (Within):               0.5343
Date:                Fri, Oct 07 2022   R-squared (Overall):              0.5270
Time:                        14:22:52   Log-likelihood                -3.463e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      2023.0
Entities:                        2180   P-value                           0.0000
Avg Obs:                      10.0000   Distribution:                F(12,21787)
Min Obs:                      10.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             2023.0
                            

In [76]:
# Random Effects 
from linearmodels.panel import RandomEffects

exog_vars = ["A", "K", "L", "year"] 
exog = sm.add_constant(df_balanced[exog_vars]) # this adds a constant 

mod = RandomEffects(df_balanced.Y, exog) # this automatically excludes dummy year = 1
re_res = mod.fit()
print(re_res)

                        RandomEffects Estimation Summary                        
Dep. Variable:                      Y   R-squared:                        0.5291
Estimator:              RandomEffects   R-squared (Between):              0.4738
No. Observations:               21800   R-squared (Within):               0.5346
Date:                Fri, Oct 07 2022   R-squared (Overall):              0.5270
Time:                        14:22:52   Log-likelihood                -3.416e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      2039.6
Entities:                        2180   P-value                           0.0000
Avg Obs:                      10.0000   Distribution:                F(12,21787)
Min Obs:                      10.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             2039.6
                            

In [77]:
# Within estimator 
from linearmodels import PanelOLS

exog_vars = ["A", "K", "L", "year"]

exog = sm.add_constant(df_balanced[exog_vars])

mod = PanelOLS(df_balanced.Y, exog, entity_effects = True, drop_absorbed=True)
fe_res = mod.fit()
print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                      Y   R-squared:                        0.5350
Estimator:                   PanelOLS   R-squared (Between):              0.4481
No. Observations:               21800   R-squared (Within):               0.5350
Date:                Fri, Oct 07 2022   R-squared (Overall):              0.5241
Time:                        14:22:53   Log-likelihood                -3.297e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      2051.1
Entities:                        2180   P-value                           0.0000
Avg Obs:                      10.0000   Distribution:                F(11,19609)
Min Obs:                      10.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             2051.1
                            

/var/folders/c9/20n96n6900d0kkmvh520dmdsj482n4/T/ipykernel_43211/2712714824.py:9: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

year.10

  fe_res = mod.fit()


In [78]:
# Between estimator 

from linearmodels.panel import BetweenOLS

exog_vars = ["A", "K", "L", "year"] 
exog = sm.add_constant(df_balanced[exog_vars])

mod = BetweenOLS(df_balanced.Y, exog)
be_res = mod.fit()
print(be_res)

                         BetweenOLS Estimation Summary                          
Dep. Variable:                      Y   R-squared:                        0.4898
Estimator:                 BetweenOLS   R-squared (Between):              0.4898
No. Observations:                2180   R-squared (Within):               0.4995
Date:                Fri, Oct 07 2022   R-squared (Overall):              0.4983
Time:                        14:22:53   Log-likelihood                   -1286.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      173.39
Entities:                        2180   P-value                           0.0000
Avg Obs:                      10.0000   Distribution:                 F(12,2167)
Min Obs:                      10.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             173.39
                            

/opt/homebrew/Caskroom/miniforge/base/envs/pp4rs/lib/python3.10/site-packages/linearmodels/panel/results.py:87: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")
/opt/homebrew/Caskroom/miniforge/base/envs/pp4rs/lib/python3.10/site-packages/linearmodels/panel/results.py:87: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")
/opt/homebrew/Caskroom/miniforge/base/envs/pp4rs/lib/python3.10/site-packages/linearmodels/panel/results.py:87: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")
/opt/homebrew/Caskroom/miniforge/base/envs/pp4rs/lib/python3.10/site-packages/linearmodels/panel/results.py:87: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")


In [79]:
# Comparing the 4 estimators 
from linearmodels.panel import compare

print(compare({"Between": be_res, "RE": re_res, "Pooled": pooled_res, "Within": fe_res}))

                                   Model Comparison                                   
                               Between                RE         Pooled         Within
--------------------------------------------------------------------------------------
Dep. Variable                        Y                 Y              Y              Y
Estimator                   BetweenOLS     RandomEffects      PooledOLS       PanelOLS
No. Observations                  2180             21800          21800          21800
Cov. Est.                   Unadjusted        Unadjusted     Unadjusted     Unadjusted
R-squared                       0.4898            0.5291         0.5270         0.5350
R-Squared (Within)              0.4995            0.5346         0.5343         0.5350
R-Squared (Between)             0.4898            0.4738         0.4762         0.4481
R-Squared (Overall)             0.4983            0.5270         0.5270         0.5241
F-statistic                     173.39     

/opt/homebrew/Caskroom/miniforge/base/envs/pp4rs/lib/python3.10/site-packages/linearmodels/panel/results.py:87: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")


In [80]:
# Hausman test of random effects vs fixed effects 
# Compare within and random effects


### Question 3
Using the balanced sub-panel compute difference estimators of the above equation. Report results from first, second and third differences. 

In [81]:
df_balanced = df_balanced.drop(columns = ["year"])
df_balanced = df_balanced.reset_index()

In [82]:
# create a differenced dataframe 
# I create a dictionary where I store the three new dataframes (first difference, second difference, third difference)
diffs = {}
for i in range(1, 4):
    df = df_balanced[["Y", "A", "K", "L", "year"]].diff(i)
    df["firm"] = df_balanced["firm"]
    year = pd.Categorical(df.year)
    df = df.set_index(['firm', 'year'])
    df["year"] = year
    diffs[i] = df

In [83]:
# First difference estimator 
diffs_estimator = {}
for i in range(1, 4):
    exog_vars = ["A", "K", "L", "year"]
    exog = sm.add_constant(diffs[i][exog_vars])
    mod = PanelOLS(diffs[i].Y, exog)
    diffs_estimator[i] = mod.fit()


/opt/homebrew/Caskroom/miniforge/base/envs/pp4rs/lib/python3.10/site-packages/linearmodels/panel/model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/opt/homebrew/Caskroom/miniforge/base/envs/pp4rs/lib/python3.10/site-packages/linearmodels/panel/model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/opt/homebrew/Caskroom/miniforge/base/envs/pp4rs/lib/python3.10/site-packages/linearmodels/panel/model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


In [85]:
print(compare({"First Diff": diffs_estimator[1], "Second Diff": diffs_estimator[2], "Third Diff": diffs_estimator[3]}))

                      Model Comparison                      
                        First Diff  Second Diff   Third Diff
------------------------------------------------------------
Dep. Variable                    Y            Y            Y
Estimator                 PanelOLS     PanelOLS     PanelOLS
No. Observations             21799        21798        21797
Cov. Est.               Unadjusted   Unadjusted   Unadjusted
R-squared                   0.4808       0.5222       0.5389
R-Squared (Within)          0.4817       0.5234       0.5408
R-Squared (Between)         0.3662       0.4466       0.4710
R-Squared (Overall)         0.4808       0.5222       0.5389
F-statistic                 5045.1       5954.1       6367.4
P-value (F-stat)            0.0000       0.0000       0.0000
===================== ============ ============ ============
const                      -0.4633       0.2337       0.2590
                         (-2.9356)     (2.2684)     (3.4598)
A                       